In [1]:
import numpy as np
import pickle
import os
from tqdm.notebook import tqdm

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search,
)

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [6]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-11-06 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-06 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-06 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-06 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-06 00:00:34,-1


In [7]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

# swmhau Network

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [8]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = True

In [9]:
num_epochs = 100
dimensions = [15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [10]:
kwargs = {
    "num_epochs": num_epochs,
    "df": anno_mi,
    "id_column": "transcript_id",
    "label_column": "client_talk_type",
    "embeddings": sbert_embeddings,
    "y_data": y_data_client,
    "output_dim": output_dim_client,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "swmhau_parameters": swmhau_parameters,
    "num_layers": num_layers,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# w=5

In [11]:
size = 5

## umap

In [12]:
(
    swmhau_network_umap_kfold_5,
    best_swmhau_network_umap_kfold_5,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_5_kfold_best_model.csv


In [13]:
swmhau_network_umap_kfold_5.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_1054464/2619527520.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_5.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.627011   
                                                                                      0.0005         0.628416   
                                                                                      0.0010         0.633269   
                                                                         0.2          0.0001         0.628480   
                                                                                      0.0005         0.636782   
                                                                                      0.0010         0.630587   
                                                          (512, 512)     0.1          0.0001         0.627395   
                                                                                      0.0005         0.629055   
                                                                                      0.0010         0.625862   
                                                                         0.2          0.0001         0.633333   
                                                                                      0.0005         0.637803   
                                                                                      0.0010         0.622031   
           12              3         10        1          (256, 256)     0.1          0.0001         0.613985   
                                                                                      0.0005         0.625990   
                                                                                      0.0010         0.625287   
                                                                         0.2          0.0001         0.616411   
                                                                                      0.0005         0.626054   
                                                                                      0.0010         0.637612   
                                                          (512, 512)     0.1          0.0001         0.621073   
                                                                                      0.0005         0.627139   
                                                                                      0.0010         0.628097   
                                                                         0.2          0.0001         0.615326   
                                                                                      0.0005         0.641443   
                                                                                      0.0010         0.619476   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.475190   
                                                                                      0.0005         0.514891   
                                                                                      0.0010         0.513896   
                                                                         0.2          0.0001         0.493314   
                                                                                      0.0005         0.513707   
                                                                                      0.0010         0.512764   
                                                          (512, 512)     0.1          0.0001         0.499493   
                                                                                      0.0005         0.513008   
                                

In [14]:
best_swmhau_network_umap_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.630651,0.510636,"[0.7540202501488983, 0.43934681181959573, 0.33...",0.505508,"[0.7771639042357275, 0.41544117647058826, 0.32...",0.517644,"[0.7322151532677849, 0.4661716171617162, 0.354...",None,0.667630,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.632375,0.506186,"[0.7589338019917984, 0.4167684167684168, 0.342...",0.502453,"[0.7688427299703264, 0.41124497991967873, 0.32...",0.510573,"[0.7492770387507229, 0.42244224422442245, 0.36]",None,0.669396,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.615134,0.496394,"[0.744060959211116, 0.41100323624595475, 0.334...",0.488891,"[0.7697063369397218, 0.4031746031746032, 0.293...",0.508828,"[0.7200694042799306, 0.41914191419141916, 0.38...",None,0.667148,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [15]:
# with speaker in path
best_swmhau_network_umap_kfold_5["f1"].mean()

0.5044055480853792

In [16]:
# no speaker in path
best_swmhau_network_umap_kfold_5["f1"].mean()

0.5044055480853792

In [17]:
# no speaker in input
best_swmhau_network_umap_kfold_5["f1"].mean()

0.5044055480853792

In [18]:
# no speaker both
best_swmhau_network_umap_kfold_5["f1"].mean()

0.5044055480853792

In [19]:
best_swmhau_network_umap_kfold_5["precision"].mean()

0.4989510919124867

In [20]:
best_swmhau_network_umap_kfold_5["recall"].mean()

0.5123483948549087

In [21]:
np.stack(best_swmhau_network_umap_kfold_5["f1_scores"]).mean(axis=0)

array([0.75233834, 0.42237282, 0.33850549])

In [22]:
np.stack(best_swmhau_network_umap_kfold_5["precision_scores"]).mean(axis=0)

array([0.77190432, 0.40995359, 0.31499537])

In [23]:
np.stack(best_swmhau_network_umap_kfold_5["recall_scores"]).mean(axis=0)

array([0.73385387, 0.43591859, 0.36727273])

# w=11

In [24]:
size = 11

## umap

In [25]:
(
    swmhau_network_umap_kfold_11,
    best_swmhau_network_umap_kfold_11,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_11_kfold_best_model.csv


In [26]:
swmhau_network_umap_kfold_11.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_1054464/404807304.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_11.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.621520   
                                                                                      0.0005         0.630204   
                                                                                      0.0010         0.622414   
                                                                         0.2          0.0001         0.637612   
                                                                                      0.0005         0.636015   
                                                                                      0.0010         0.632886   
                                                          (512, 512)     0.1          0.0001         0.617369   
                                                                                      0.0005         0.625990   
                                                                                      0.0010         0.620690   
                                                                         0.2          0.0001         0.621520   
                                                                                      0.0005         0.639719   
                                                                                      0.0010         0.630460   
           12              3         10        1          (256, 256)     0.1          0.0001         0.614496   
                                                                                      0.0005         0.640166   
                                                                                      0.0010         0.629374   
                                                                         0.2          0.0001         0.621264   
                                                                                      0.0005         0.623627   
                                                                                      0.0010         0.623116   
                                                          (512, 512)     0.1          0.0001         0.611367   
                                                                                      0.0005         0.632759   
                                                                                      0.0010         0.619349   
                                                                         0.2          0.0001         0.618199   
                                                                                      0.0005         0.641762   
                                                                                      0.0010         0.627395   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.434046   
                                                                                      0.0005         0.515692   
                                                                                      0.0010         0.504768   
                                                                         0.2          0.0001         0.480857   
                                                                                      0.0005         0.516660   
                                                                                      0.0010         0.508953   
                                                          (512, 512)     0.1          0.0001         0.468130   
                                                                                      0.0005         0.503812   
                                

In [27]:
best_swmhau_network_umap_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.623180,0.505709,"[0.7506694436179708, 0.41566014160766346, 0.35...",0.497944,"[0.7729779411764706, 0.41968040370058873, 0.30...",0.520443,"[0.7296124927703875, 0.41171617161716173, 0.42]",None,0.667148,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.634483,0.523777,"[0.7546366676801459, 0.47072419106317415, 0.34...",0.514284,"[0.7955128205128205, 0.44147398843930635, 0.30...",0.540021,"[0.7177559282822441, 0.5041254125412541, 0.398...",None,0.666506,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.632950,0.517590,"[0.7568134171907758, 0.4432044639298525, 0.352...",0.509277,"[0.7847826086956522, 0.42868157286044717, 0.31...",0.530444,"[0.7307692307692307, 0.45874587458745875, 0.40...",None,0.663616,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [28]:
best_swmhau_network_umap_kfold_11["f1"].mean()

0.5156922830494409

In [29]:
best_swmhau_network_umap_kfold_11["precision"].mean()

0.5071684065132042

In [30]:
best_swmhau_network_umap_kfold_11["recall"].mean()

0.5303027900630819

In [31]:
np.stack(best_swmhau_network_umap_kfold_11["f1_scores"]).mean(axis=0)

array([0.75403984, 0.44319627, 0.34984074])

In [32]:
np.stack(best_swmhau_network_umap_kfold_11["precision_scores"]).mean(axis=0)

array([0.78442446, 0.42994532, 0.30713544])

In [33]:
np.stack(best_swmhau_network_umap_kfold_11["recall_scores"]).mean(axis=0)

array([0.72604588, 0.45819582, 0.40666667])

# w=20

In [34]:
size = 20

## umap

In [35]:
(
    swmhau_network_umap_kfold_20,
    best_swmhau_network_umap_kfold_20,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_20_kfold_best_model.csv


In [36]:
swmhau_network_umap_kfold_20.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_1054464/1154838213.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_20.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.620626   
                                                                                      0.0005         0.642146   
                                                                                      0.0010         0.630460   
                                                                         0.2          0.0001         0.616922   
                                                                                      0.0005         0.643806   
                                                                                      0.0010         0.637484   
                                                          (512, 512)     0.1          0.0001         0.605045   
                                                                                      0.0005         0.634483   
                                                                                      0.0010         0.617752   
                                                                         0.2          0.0001         0.609515   
                                                                                      0.0005         0.635313   
                                                                                      0.0010         0.624393   
           12              3         10        1          (256, 256)     0.1          0.0001         0.598212   
                                                                                      0.0005         0.626820   
                                                                                      0.0010         0.629119   
                                                                         0.2          0.0001         0.596679   
                                                                                      0.0005         0.625287   
                                                                                      0.0010         0.627714   
                                                          (512, 512)     0.1          0.0001         0.606322   
                                                                                      0.0005         0.633716   
                                                                                      0.0010         0.630907   
                                                                         0.2          0.0001         0.611111   
                                                                                      0.0005         0.635377   
                                                                                      0.0010         0.630779   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.426151   
                                                                                      0.0005         0.525420   
                                                                                      0.0010         0.512395   
                                                                         0.2          0.0001         0.463470   
                                                                                      0.0005         0.518649   
                                                                                      0.0010         0.513623   
                                                          (512, 512)     0.1          0.0001         0.448430   
                                                                                      0.0005         0.514204   
                                

In [37]:
best_swmhau_network_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.640613,0.518225,"[0.7704314533941983, 0.4176013805004314, 0.366...",0.511637,"[0.7848784878487849, 0.4376130198915009, 0.312...",0.533161,"[0.7565066512434934, 0.39933993399339934, 0.44...",None,0.666024,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.620690,0.505718,"[0.7463626818659067, 0.42158385093167694, 0.34...",0.498814,"[0.7753194141477096, 0.3980938416422287, 0.323...",0.515837,"[0.719491035280509, 0.44801980198019803, 0.38]",None,0.671002,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.639847,0.520701,"[0.7626529687263937, 0.47174819871065604, 0.32...",0.513655,"[0.7984814931983549, 0.43649122807017543, 0.30...",0.531943,"[0.7299016772700984, 0.5132013201320133, 0.352...",None,0.667309,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [38]:
best_swmhau_network_umap_kfold_20["f1"].mean()

0.5148812454309205

In [39]:
best_swmhau_network_umap_kfold_20["precision"].mean()

0.5080356129275914

In [40]:
best_swmhau_network_umap_kfold_20["recall"].mean()

0.5269804506959276

In [41]:
np.stack(best_swmhau_network_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.7598157 , 0.43697781, 0.34785022])

In [42]:
np.stack(best_swmhau_network_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.78622647, 0.42406603, 0.31381434])

In [43]:
np.stack(best_swmhau_network_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.73529979, 0.45352035, 0.39212121])

# w=35

In [44]:
size = 35

## umap

In [ ]:
(
    swmhau_network_umap_kfold_35,
    best_swmhau_network_umap_kfold_35,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
swmhau_network_umap_kfold_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_35

In [ ]:
best_swmhau_network_umap_kfold_35["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_35["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_35["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["recall_scores"]).mean(axis=0)

# w=80

In [ ]:
size = 80

## umap

In [ ]:
(
    swmhau_network_umap_kfold_80,
    best_swmhau_network_umap_kfold_80,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_80

In [ ]:
best_swmhau_network_umap_kfold_80["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_80["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["recall_scores"]).mean(axis=0)

## w=110

In [ ]:
size = 110

## umap

In [ ]:
(
    swmhau_network_umap_kfold_110,
    best_swmhau_network_umap_kfold_110,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_5.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_20

In [ ]:
best_swmhau_network_umap_kfold_20["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_20["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_20["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_20["recall_scores"]).mean(axis=0)